In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pickle
import random

rng = np.random.default_rng(926334)

In [4]:
ROOT = 'C:/Users/Matt/Documents/Python Scripts/SnowComp/realtime/'
MODIS_CNN_PATH = ROOT + 'model_32_18_8_3_0.13_50_1399_0.0001' #'modis_model'

import sys
sys.path.append(ROOT)

import torch
import captum.attr as attrib
from modis_cnn import Net as ModisNet

net_kwargs = {'cdim1': 32,
              'cdim2': 18,
              'cdim3': 8,
              'kernel_sz': 3,
              'dropout': 0.13,
              'ldim': 50}


modis_net = ModisNet(**net_kwargs)
modis_net.load_state_dict(torch.load(MODIS_CNN_PATH))

c1 dim: 19
mp0 dim: 17
c2 dim: 15
mp1 dim: 13
c3 dim: 11
mp2 dim: 9
flattened_dim 648


<All keys matched successfully>

## Make dropout predictions and estimations of uncertainty

In [2]:
#@title Get Modis CNN predictions
def dout_predict(cnn, x, as_numpy=False):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    cnn.train()
    print("model is in Training mode")
    
    x = x.type(torch.FloatTensor)
    
    vals_sub = []
    
    x = TensorDataset(x)
    sub_loader = DataLoader(x, batch_size=5000)
    with torch.no_grad():
        for images in sub_loader:
            images = images[0].to(device)
            vals_sub.append(cnn(images).cpu().numpy())
            
    vals_sub = np.concatenate(vals_sub, axis = 0)    
    
    return vals_sub.squeeze()